# Lesson 4 - Wrapping the RAG Agent into an ACP Server

In this lesson, you will wrap the RAG CrewAI agent you created in the last lesson in ACP server and then run the ACP server to activate the agent so it can be discoverable by an ACP client.

## 4.1. Wrap the Agent in ACP  Server

You will now take the same code you worked on in Lesson 3 and wrap it in a python file called: `crew_agent_server`.

To make the agent ACP compliant, you can use the `@server.agent()` decorator to define your agent. The name is inferred from the function name, and the description is pulled from the docstring. Here's the minimal structure needed for an ACP-compliant agent:

```python
@server.agent()
async def policy_agent(input: list[Message]) -> AsyncGenerator[RunYield, RunYieldResume]:
    "This is an agent for questions around policy coverage, it uses a RAG pattern to find answers based on policy documentation. Use it to help answer questions on coverage and waiting periods."
    # Here goes the function definition
    # ....
    task_output = ...
    yield Message(parts=[MessagePart(content=str(task_output))])
```
This configuration establishes several critical aspects of the agent:
- **Function Definition**: The core functionality that determines what the agent does;
- **Input Parameter**: The input parameter accepts a list of Message objects; 
- **Return Type**: The AsyncGenerator[RunYield, RunYieldResume] return type enables both streaming responses and the await pattern:
   - AsyncGenerator: An async generator object that can be iterated with async for and supports await operations
   - RunYield: The type of values this generator yields (sends out)
   - RunYieldResume: The type of values this generator receives when resumed (sent back in) (in the definition below, you will only use RunYield)
- **Documentation**: The docstring provides a human-readable description of the agent

Run the following cell to copy the content of the cell to the file `crew_agent_server.py` which will be saved under the folder `my_acp_project`. 

<p style="background-color:#fff6ff; padding:15px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px"> 💻 &nbsp; <b>To access the <code>my_acp_project</code> folder:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Open"</em>. 

In [ ]:
%%writefile ../my_acp_project/crew_agent_server.py
from collections.abc import AsyncGenerator
from acp_sdk.models import Message, MessagePart
from acp_sdk.server import RunYield, RunYieldResume, Server

from crewai import Crew, Task, Agent, LLM
from crewai_tools import RagTool

import nest_asyncio
nest_asyncio.apply()

server = Server()
llm = LLM(model="openai/gpt-4", max_tokens=1024)

config = {
    "llm": {
        "provider": "openai",
        "config": {
            "model": "gpt-4",
        }
    },
    "embedding_model": {
        "provider": "openai",
        "config": {
            "model": "text-embedding-ada-002"
        }
    }
}
rag_tool = RagTool(config=config,  
                   chunk_size=1200,       
                   chunk_overlap=200,     
                  )
rag_tool.add("../data/gold-hospital-and-premium-extras.pdf", data_type="pdf_file")


@server.agent()
async def policy_agent(input: list[Message]) -> AsyncGenerator[RunYield, RunYieldResume]:
    "This is an agent for questions around policy coverage, it uses a RAG pattern to find answers based on policy documentation. Use it to help answer questions on coverage and waiting periods."

    insurance_agent = Agent(
        role="Senior Insurance Coverage Assistant", 
        goal="Determine whether something is covered or not",
        backstory="You are an expert insurance agent designed to assist with coverage queries",
        verbose=True,
        allow_delegation=False,
        llm=llm,
        tools=[rag_tool], 
        max_retry_limit=5
    )
    
    task1 = Task(
         description=input[0].parts[0].content,
         expected_output = "A comprehensive response as to the users question",
         agent=insurance_agent
    )
    crew = Crew(agents=[insurance_agent], tasks=[task1], verbose=True)
    
    task_output = await crew.kickoff_async()
    yield Message(parts=[MessagePart(content=str(task_output))])

if __name__ == "__main__":
    server.run(port=8001)

## 4.2. Run the Insurer ACP Server

Now to activate your configured ACP agent, you would need to run your agent server. The folder `my_acp_project` has been set up for you so you can run the agent server using `uv`:

- Open the terminal by running the cell below
- Type `uv run crew_agent_server.py` to run the server and activate your ACP agent.

Please see note below if you want to replicate the work locally on your machine.

In [ ]:
from IPython.display import IFrame
import os
url = os.environ.get('DLAI_LOCAL_URL').format(port=8888)
IFrame(f"{url}terminals/1", width=800, height=600)

You now have an agent running on port 8001 that can receive messages from others, or be called via HTTP, using the ACP protocol. 

**Note**: If you see this warning: 
`WARNING: Can not reach server, check if running on http://127.0.0.1:8333 : Request failed after 5 retries`
you can ignore it. You'll learn later in another lesson about the BeeAI platform, which a registry you can use to manage and discover agents. If the platform is installed, it runs by default on port 8333. The ACP servers are configured to automatically connect to the platform. Since the platform is not installed in this environment, the ACP server will generate a warning.

**Note: How to set up `my_acp_project` locally on your machine using the `uv` tool?**

- First install `uv` by checking this [link](https://docs.astral.sh/uv/getting-started/installation/).

After that, you can create `my_acp_project` in any directory of your choice, then in the terminal you can type the following commands:
- `cd my_acp_porject`
- `uv init`: to initialize the project
- `uv venv`: to create a virtual environment
- `uv add crewai crewai-tools acp-sdk load_dotenv nest-asyncio`: to define the dependencies.

Then create `crew_agent_server.py` inside the `my_acp_project`.

You can then run the server using `uv run`.  Since this code uses an OpenAI model, you would also need to specify an openAI API key in a `.env` file like this: `OPENAI_API_KEY=sk-...`. If you would like to use a local open source model using `Ollama`, please check the resource section below.

## Resources

- [How to wrap Agent](https://agentcommunicationprotocol.dev/how-to/wrap-existing-agent)
- [Configuration of ACP Agent](https://agentcommunicationprotocol.dev/core-concepts/agent-lifecycle#configuration)
- [Same code using a local open source model: `ollama_chat/qwen2.5:14b`](https://github.com/nicknochnack/ACPWalkthrough/blob/main/2.%20CrewAI%20via%20Server.py)

<div style="background-color:#fff6ff; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
<p> ⬇ &nbsp; <b>Download Notebooks:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Download as"</em> and select <em>"Notebook (.ipynb)"</em>.</p>

</div>